# Advent of Code - Day 10

In [ ]:
from itertools import chain
with open('input.txt', 'r') as file:
    lines = [line.strip().split() for line in file]

for i, line in enumerate(lines):
    lines[i] = list(str(line).replace("[","").replace("]","").replace("'",""))


all_val = list(set(list(chain.from_iterable(lines))))


for d in lines:
    d.insert(0, "X")
    d.insert(len(d),"X")

dum = list("X"*len(lines[0]))
lines.extend([dum])
lines.insert(0, dum)

In [ ]:
# find index of S
break_flag = False
for iy, y in enumerate(lines):
    for ix, x in enumerate(y):
        if x == "S":
            break_flag = True
        if break_flag:
            break
    if break_flag:
            break

coords = [[ix, iy]]

## Part One

In [ ]:
def find_next(xco, yco, oneswitch, twoswitch, threeswitch, fourswitch):
    
    
    box = [line[xco-1:xco+2] for line in lines[yco-1:yco+2]]
    next_poss =  list(chain.from_iterable(box))
    
    # Indices of elements to remove
    indices_to_remove = [0, 2, 4, 6, 8]
    
    for index in sorted(indices_to_remove, reverse=True):
        del next_poss[index]  
    
    for i, val in enumerate(next_poss):
        # Test One
        if i == 0 and val in [all_val[1], all_val[-1], all_val[-2], all_val[4]] and oneswitch:
            coord_new = [xco,yco-1]
            if coord_new not in coords:
                coords.append(coord_new)
                break
        # Test Two
        if i == 1 and val in [all_val[2], all_val[5], all_val[-2], all_val[4]] and twoswitch:
            coord_new = [xco-1,yco]
            if coord_new not in coords:
                coords.append(coord_new)
                break
        # Test Three
        if i == 2 and val in [all_val[-2], all_val[0], all_val[-1], all_val[2]] and threeswitch:
            coord_new = [xco+1,yco]
            if coord_new not in coords:
                coords.append(coord_new)
                break
        # Test Four
        if i == 3 and val in [all_val[-2], all_val[1], all_val[-3], all_val[0]] and fourswitch:
            coord_new = [xco,yco+1]
            if coord_new not in coords:
                coords.append(coord_new)
                break
    return(coord_new)

In [ ]:
oneswitch = True; twoswitch = True; threeswitch = True; fourswitch = True; s_flag = True
coord_breaker = []


while s_flag :
    
    new_coords = find_next(coords[-1][0], coords[-1][1], oneswitch, twoswitch, threeswitch, fourswitch)
    
    item = lines[coords[-1][1]][coords[-1][0]]

    # By default switches are set to FALSE
    oneswitch = False; twoswitch = False; threeswitch = False; fourswitch = False
    
    # Does pipe connect upwards
    if item in [all_val[1], all_val[5], all_val[0]]:
        oneswitch = True
    # Does pipe connect west
    if item in [all_val[0], all_val[-1], all_val[2]]:
        twoswitch = True
    # Does pipe connect east
    if item in [all_val[5], all_val[4], all_val[2]]:
        threeswitch = True
    # Does pipe connect downwards
    if item in [all_val[1], all_val[7], all_val[4]]:
        fourswitch = True
    
    # Are we back to the start
    if new_coords in coord_breaker:
        s_flag = False
        
    coord_breaker.append(new_coords)

print(len(coords)/2)

## Part Two

This part requires use of photo editing software, decided this was not cheating<br>
After chunk 10, open up your .png in MS Paint and flood fill the bit clearly on the outside with white<br>
We can then find values which were not filled<br>

In [ ]:
import copy 

toney = copy.deepcopy(lines)
for y, v1 in enumerate(toney):
    for x, v2 in enumerate(v1):
        if [x,y] not in coords:
            toney[y][x] = 0

In [ ]:
def get_cell(value):
    if value == 0:
        cell = [[0,0,0],
                [0,90,0],
                [0,0,0]]
    if value == 'S':
        cell = [[0,0,0],
                [0,255,255],
                [0,255,0]]
    if value == '|':
        cell = [[0,255,0],
                [0,255,0],
                [0,255,0]]
    if value == 'L':
        cell = [[0,255,0],
                [0,255,255],
                [0,0,0]]
    if value == 'J':
        cell = [[0,255,0],
                [255,255,0],
                [0,0,0]]
    if value == '7':
        cell = [[0,0,0],
                [255,255,0],
                [0,255,0]]
    if value == '-':
        cell = [[0,0,0],
                [255,255,255],
                [0,0,0]]
    if value == 'F':
        cell = [[0,0,0],
                [0,255,255],
                [0,255,0]]
    return cell

In [ ]:
flattened_list = list(chain.from_iterable(toney))

maze = []
for i in flattened_list:
    maze.append(get_cell(i))
    
# Number of chunks
num_chunks = 142

# Calculate the size of each chunk
chunk_size = len(maze) // num_chunks

# Divide the list into chunks
chunks = [maze[i:i+chunk_size] for i in range(0, len(maze), chunk_size)]


maze1 = []
for chunk in chunks:
    for val in range(3):
        new_chunk = list(chain.from_iterable([chunk1[val] for chunk1 in chunk]))
        maze1.append(new_chunk)
        del new_chunk

from PIL import Image

# Example list of lists representing pixel data (grayscale image)
pixel_data = maze1

# Create a new image with PIL
image = Image.new("RGB", (len(pixel_data[0]), len(pixel_data)))

# Set pixel values based on the list of lists
for y, row in enumerate(pixel_data):
    for x, pixel_value in enumerate(row):
        image.putpixel((x, y), (pixel_value, pixel_value, pixel_value))  # Grayscale image

# Save the image as a PNG file
image.save("output.png")


### MS Paint here

In [ ]:
# Get the width and height of the image
image = Image.open("output.png")
width, height = image.size

# Create an empty list to store the green values of all pixels
green_values_list = []

# Iterate over all pixels in the image
for y in range(height):
    row_green_values = []  # Empty list to store green values of pixels in a row
    for x in range(width):
        rgb_values = image.getpixel((x, y))  # Get RGB values of the pixel at (x, y)
        green_value = rgb_values[1]  # Extract green value (index 1) from RGB tuple
        row_green_values.append(green_value)   # Append green value to the row list
    green_values_list.append(row_green_values)  # Append the row list to the main list

In [ ]:
import statistics

counter = 0
for i in range(1, len(green_values_list)+1, 3):
    row = green_values_list[i]
    for i2 in range(1, len(row)+1, 3):
        if row[i2] == 90:
            box1 = []
            for box in green_values_list[i-1:i+2]:
                box1.extend(box[(i2-1):(i2+2)])
            if statistics.median(box1) == 0:
                counter  += 1 
print(counter)